## Instruction
1. Comparision for batch of images, need to specify the path

In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## 0. Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image

import sys
sys.path.append("../../../src/")
sys.path.append("../../")
import model
from datasets import imagenet
from loss import FileterLoss
import config
#from utils.function import init_logging, init_environment, preprocess_image,\
#         recreate_image, get_lr, save_image
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map_for_comparision

In [3]:
# Load data
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
reverse_mean = [-0.485, -0.456, -0.406]
reverse_std = [1/0.229, 1/0.224, 1/0.225]
train_transform = None
train_transform = transforms.Compose([                                                                                                                                
    transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
    transforms.ToTensor(),                                                                                                                                            
    transforms.Normalize(mean, std)                                                                                                                                   
       ])         
trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", is_train=True, transform=train_transform)

trainset.set_data([950], 30)
imgs_path = []                                                                                                                                                            
images = []
labels = []
for img, label, img_path in trainset:                                                                                                                                     
    images.append(img.unsqueeze(0))                                                                                                                                       
    labels.append(label)                                                                                                                                                  
    imgs_path.append(img_path)  

Len of new dataset is :30


In [4]:
save_dir = "/home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack"
################### Hyper-Parameter #######################
exp = "2204"
epoch = "49900"
##########################################################
ab_path = os.path.join(save_dir, exp, epoch)

In [5]:
resume = "037-0"
model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
backbone = "vgg16"

# Load model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = model.Network(backbone=backbone, num_classes=1000)
net.to(device)
# resume from model
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt) 

> Use original fc
Resume from model from exp: 037 at epoch 0


<All keys matched successfully>

In [6]:
# Load image
optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True)
valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
optimized_data = zscore(optimized_data, mean, std)

Load from /home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack/2204/49900/n07747607_10061.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack/2204/49900/n07747607_10068.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack/2204/49900/n07747607_10110.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack/2204/49900/n07747607_10134.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack/2204/49900/n07747607_10162.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack/2204/49900/n07747607_10166.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack/2204/49900/n07747607_10178.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack/2204/49900/n07747607_10179.JPEG
Load from /home/lincolnzjx/Desktop/Interpretation/jupyter/analysis/0407/pack/2204/49900/n07747607_10184.JPEG
Load from /home/lin

In [7]:
# Move to device
opt_image = torch.from_numpy(optimized_data).to(device)
original_image = torch.cat(valid_imgs, dim=0).to(device)

In [8]:
selected_layer = 6
selected_filter = 19
mode = "keep"
inter = False
rho = 0
regularization = "L1"
smoothing = "None"
regular_ex = 2
alpha = 100
beta = 1
gamma = 100
delta = 0

In [9]:
# 2.Calculate Loss
criterion = FileterLoss(net, selected_layer, selected_filter, mode,
                        inter=inter, rho=rho,
                        regularization=regularization,
                        smoothing=smoothing, p=regular_ex, _print=print)

In [10]:
selected_filter_loss, rest_fileter_loss, regularization_loss,  \
      smoothing_loss = criterion(opt_image, original_image)
loss = alpha * selected_filter_loss + beta * rest_fileter_loss + \
    gamma * regularization_loss + delta * smoothing_loss
print(selected_filter_loss)
print(rest_fileter_loss)
print(regularization_loss)
print(smoothing_loss)

tensor(2.4978, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3954, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.1260, device='cuda:0')
tensor(0)
